Process lightcurves produced by the FPBot and saved to some directory. 

In [ ]:
import sys, os

In [ ]:
from ampel.secret.AmpelVault import AmpelVault
from ampel.secret.DictSecretProvider import DictSecretProvider
from ampel.dev.DevAmpelContext import DevAmpelContext
from ampel.alert.load import DirAlertLoader
from ampel.alert.load import FileAlertLoader


In [ ]:
AMPEL_CONF = '/home/jnordin/github/ampel83/Ampel-HU-astro/ampel_conf.yaml'

In [ ]:
# Create a secret vault
secrets = { 'slack/ztf_general/jno': os.environ["SNIaSLACK"],
                              'mongo/logger':{},
                              'mongo/writer':{},
                              'extcats/reader':{},
            'ztf/archive/auth' : {'user' : '', 'password' : ''},
            'datalab/pwd':os.environ["DATALABPWD"],
            "desycloud" : {},
            "ztf/archive/token": os.environ["ARCHIVE_TOKEN"],
          }
vault = AmpelVault([DictSecretProvider(secrets)])

In [ ]:
# Select date of query with format e.g. '20210610'
channel = 'test'
ITER_MAX = 1000000
ALERT_DIR = '/home/jnordin/data/voids/fpbot'
#fpath = '/home/jnordin/data/ztf/simeon_cloud/data/0-999_cleaned/ZTF18aacaosx.csv'

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'dumpme',
    purge_db = True,
    vault = vault
)
ctx.add_channel(
    name=channel,
    access=['ZTF', 'ZTF_PUB', 'ZTF_PRIV']
)

In [ ]:
directives = [
        {
            "channel": channel,
            "ingest": {
                "mux": {
                    "unit": "ZiMongoMuxer",
                    "combine": [
                        {
                            "unit": "ZiT1Combiner",
                        }
                    ],
                }
            }
        }
    ]

In [ ]:
ac = ctx.new_context_unit(
    unit = "AlertConsumer",
    process_name = "AP_test",
    iter_max = ITER_MAX,
    log_profile = os.environ.get('log_profile', 'debug'),
    shaper = "ZiDataPointShaper",
    compiler_opts = "ZiCompilerOptions",
    supplier = {
        "unit": "ZTFFPbotForcedPhotometryAlertSupplier",
        'config': {
            'deserialize': "csv",
            'loader': {
                'unit': 'DirAlertLoader',
                'config': {'folder': ALERT_DIR, 'extension' : '*.csv'}
            }
        }
    },
    directives = directives
)


In [ ]:
n = ac.run()

In [ ]:
print(f"{n} alerts processed for channel {channel}")

In [ ]:
t2w = ctx.new_context_unit(
    unit = 'T2Worker',
    process_name = 'T2Processor_test',
    log_profile = os.environ.get('log_profile', 'default')
)

In [ ]:
t2w.run()

In [ ]:
table_config =  {
                                        'table_schema' : {
                                            'T2DigestRedshifts': {
                                                'Ampel z' : ['ampel_z'],
                                                'Ampel z group' : ['group_z_nbr'],
                                            },
                                            'T2RunSncosmo' : {
                                                'Peak brightness (abs mag)' : ['fit_metrics','restpeak_model_absmag_B'],
                                                'SALT2 X1' : ['sncosmo_result','paramdict','x1'],
                                                'SALT2 X1 (err)' : ['sncosmo_result','errors','x1'],
                                                'SALT2 Color' : ['sncosmo_result','paramdict','c'],
                                                'SALT2 Color (err)' : ['sncosmo_result','errors','c'],
                                                'Pulls around peak' : ['fit_metrics','absmean_peak_pull'],
                                                'Det. around peak' : ['fit_metrics','nbr_peak_pulls'],
                                            },
                                        },
                                        'transient_table_schema' : {
                                            'T2CatalogMatch': {
                                                'SDSS spec z' : ['SDSS_spec','z'],
                                                "NED z" : ['NEDz','z'],
                                                "NED dist" : ['NEDz','dist2transient'],
                                                "Glade z" : ['GLADEv23','z'],
                                                "LS_Zou Photo-z": ['LSPhotoZZou','photoz'],
                                                "LS_Zou dist": ['LSPhotoZZou','dist2transient'],
                                                "WiseCos Photo-z": ['wiseScosPhotoz','zPhoto_Corr'],
                                                "WiseCos dist": ['wiseScosPhotoz','dist2transient'],
                                            },
                                        },
                                        'include_stock' : True,
                                        'include_channels' : True,
                                        'fmt' : 'csv',
                                        'local_path' : '/home/jnordin/tmp'
                                    }

In [ ]:
t3p = ctx.new_context_unit(
    process_name = "LensPrint",
    unit = "T3Processor",
    execute = [
        {
            "unit": "T3ReviewUnitExecutor",
            "config": {
                "supply": {
                    "unit": "T3DefaultBufferSupplier",
                    "config": {
                        "select": {
                            "unit": "T3StockSelector",
                            "config": {"channel": channel}
                        },
                        "load": {
                            "unit": "T3SimpleDataLoader",
                            "config": {
                                "directives": ["STOCK", "T1", "T2DOC"],
                                "channel": channel
                            }
                        }
                     }
                },
                "stage" : {
                    "unit": "T3SimpleStager",
                    "config": {
                        "execute": [
                            {
                                "unit": "TransientTablePublisher",
                                "config": table_config
                            }
                        ]
                    }
                }
            }
        }
    ]
)

In [ ]:
t3p.run()